# Merge metadata with ESCO Crosswalk
Felix Zaussinger | 18.05.2021

## Core Analysis Goal(s)
1. merge onet greenness scores and ASHE wage & employment data to onet-esco crosswalk on isco 4-digit level

## Key Insight(s)
1. None

In [100]:
import os
import sys
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("ticks")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define directory structure

In [101]:
# project directory
abspath = os.path.abspath('')
project_dir = str(Path(abspath).parents[0])

# sub-directories
data_raw = os.path.join(project_dir, "data", "raw")
data_interim = os.path.join(project_dir, "data", "interim")
data_processed = os.path.join(project_dir, "data", "processed")
figure_dir = os.path.join(project_dir, "reports", "figures")

Read crosswalk

In [102]:
crosswalk = pd.read_csv(
    os.path.join(data_raw, "onet", "onet_esco_crosswalk", "processed",
                 "ESCO_ONET_xwalk_full.csv")
)

# decompose isco 4-digit level
crosswalk["isco_level_1"] = crosswalk["isco_level_4"].astype(str).str[:1].astype(int)
crosswalk["isco_level_2"] = crosswalk["isco_level_4"].astype(str).str[:2].astype(int)
crosswalk["isco_level_3"] = crosswalk["isco_level_4"].astype(str).str[:3].astype(int)

Read greenness data

In [104]:
greenness = pd.read_excel(
    io=os.path.join(data_raw, "onet", "Onet_GreenTask_AppA.xlsx"),
    sheet_name="Occupations"
)

Read ASHE employment and earnings data (at 4-digit level)

In [105]:
uk_employment = pd.read_csv(
    os.path.join(data_raw, "onet", "onet_esco_crosswalk", "processed",
                 "linked_data", "ESCO_top_occupations_UK_employment.csv")
)

uk_employment[["isco_level_4", "employment_share", "employment_count"]]

,isco_level_4,employment_share,employment_count
0,8121,0.000020,645.0
1,3155,0.000087,2797.0
2,2431,0.000648,20834.0
3,3212,0.000647,20808.0
4,5312,0.004302,138321.0
...,...,...,...
1696,8131,0.000053,1699.0
1697,2120,0.000286,9184.0
1698,9612,0.000417,13414.0
1699,2422,0.001128,36261.0


In [106]:
uk_earnings = pd.read_csv(
    os.path.join(data_raw, "onet", "onet_esco_crosswalk", "processed",
                 "linked_data", "ESCO_occupations_UK_earnings_and_hours_imputed.csv")
)

uk_earnings[["isco_level_4", "total_paid_hours", "annual_earnings"]]

,isco_level_4,total_paid_hours,annual_earnings
0,2166,36.670202,37555.308176
1,8121,39.904110,22031.963636
2,7543,38.709386,22430.367647
3,3155,42.036716,32754.000000
4,2431,36.354676,38839.816377
...,...,...,...
2937,2120,35.494487,36135.290249
2938,9612,39.443972,21357.054545
2939,5414,37.021429,24315.540984
2940,2422,35.240566,33292.986333


read onet job zone data

In [115]:
job_zones = pd.read_csv(
    os.path.join(data_raw, "onet", "onet_esco_crosswalk", "processed",
                 "linked_data", "ESCO_occupations_Job_Zones.csv")
)

job_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2942 entries, 0 to 2941
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2942 non-null   int64  
 1   concept_uri              2942 non-null   object 
 2   preferred_label          2942 non-null   object 
 3   isco_level_4             2942 non-null   int64  
 4   onet_code                2921 non-null   object 
 5   onet_occupation          2921 non-null   object 
 6   O*NET-SOC Code           2860 non-null   object 
 7   job_zone                 2860 non-null   float64
 8   education_level          2858 non-null   float64
 9   related_work_experience  2858 non-null   float64
 10  on_the_job_training      2858 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 253.0+ KB


left-join greenness index

In [107]:
df_merged_1 = pd.merge(
    left=crosswalk,
    right=greenness,
    left_on="onet_code",
    right_on="Code",
    how="left",
    validate="many_to_one"
)

df_merged_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2942 entries, 0 to 2941
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2942 non-null   int64  
 1   concept_uri             2942 non-null   object 
 2   preferred_label         2942 non-null   object 
 3   isco_level_4            2942 non-null   int64  
 4   onet_code               2921 non-null   object 
 5   onet_occupation         2921 non-null   object 
 6   isco_level_1            2942 non-null   int64  
 7   isco_level_2            2942 non-null   int64  
 8   isco_level_3            2942 non-null   int64  
 9   Code                    432 non-null    object 
 10  Title                   432 non-null    object 
 11  Occupation Type         432 non-null    object 
 12  # New Green Tasks       432 non-null    float64
 13  # Existing Green Tasks  432 non-null    float64
 14  # Non-Green Tasks       432 non-null    

left-join earnings data

In [108]:
df_merged_2 = pd.merge(
    left=crosswalk,
    right=uk_earnings[["isco_level_4", "total_paid_hours", "annual_earnings"]],
    on="isco_level_4",
    how="left"
).drop_duplicates().reset_index(drop=True)

df_merged_2

,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_occupation,isco_level_1,isco_level_2,isco_level_3,total_paid_hours,annual_earnings
0,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176
1,1,http://data.europa.eu/esco/occupation/000e93a3...,metal drawing machine operator,8121,51-4021.00,"extruding and drawing machine setters, operato...",8,81,812,39.904110,22031.963636
2,2,http://data.europa.eu/esco/occupation/0019b951...,precision device inspector,7543,51-9061.00,"inspectors, testers, sorters, samplers, and we...",7,75,754,38.709386,22430.367647
3,3,http://data.europa.eu/esco/occupation/0022f466...,air traffic safety technician,3155,17-3023.01,electronics engineering technicians,3,31,315,42.036716,32754.000000
4,4,http://data.europa.eu/esco/occupation/002da35b...,hospitality revenue manager,2431,13-1161.00,market research analysts and marketing special...,2,24,243,36.354676,38839.816377
...,...,...,...,...,...,...,...,...,...,...,...
2937,2937,http://data.europa.eu/esco/occupation/ff656b3a...,demographer,2120,15-2041.00,statisticians,2,21,212,35.494487,36135.290249
2938,2938,http://data.europa.eu/esco/occupation/ff8d4065...,sorter labourer,9612,51-9199.01,recycling and reclamation workers,9,96,961,39.443972,21357.054545
2939,2939,http://data.europa.eu/esco/occupation/ffa4dd5d...,armoured car guard,5414,33-9032.00,security guards,5,54,541,37.021429,24315.540984
2940,2940,http://data.europa.eu/esco/occupation/ffade2f4...,civil service administrative officer,2422,11-3011.00,administrative services managers,2,24,242,35.240566,33292.986333


left-join employment data

In [119]:
df_merged_3 = pd.merge(
    left=df_merged_2,
    right=uk_employment[["isco_level_4", "employment_share", "employment_count"]],
    on="isco_level_4",
    how="left"
).drop_duplicates().reset_index(drop=True)

df_merged_3

,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_occupation,isco_level_1,isco_level_2,isco_level_3,total_paid_hours,annual_earnings,employment_share,employment_count
0,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0
1,1,http://data.europa.eu/esco/occupation/000e93a3...,metal drawing machine operator,8121,51-4021.00,"extruding and drawing machine setters, operato...",8,81,812,39.904110,22031.963636,0.000020,645.0
2,2,http://data.europa.eu/esco/occupation/0019b951...,precision device inspector,7543,51-9061.00,"inspectors, testers, sorters, samplers, and we...",7,75,754,38.709386,22430.367647,0.000228,7338.0
3,3,http://data.europa.eu/esco/occupation/0022f466...,air traffic safety technician,3155,17-3023.01,electronics engineering technicians,3,31,315,42.036716,32754.000000,0.000087,2797.0
4,4,http://data.europa.eu/esco/occupation/002da35b...,hospitality revenue manager,2431,13-1161.00,market research analysts and marketing special...,2,24,243,36.354676,38839.816377,0.000648,20834.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,2937,http://data.europa.eu/esco/occupation/ff656b3a...,demographer,2120,15-2041.00,statisticians,2,21,212,35.494487,36135.290249,0.000286,9184.0
2938,2938,http://data.europa.eu/esco/occupation/ff8d4065...,sorter labourer,9612,51-9199.01,recycling and reclamation workers,9,96,961,39.443972,21357.054545,0.000417,13414.0
2939,2939,http://data.europa.eu/esco/occupation/ffa4dd5d...,armoured car guard,5414,33-9032.00,security guards,5,54,541,37.021429,24315.540984,0.001450,46635.0
2940,2940,http://data.europa.eu/esco/occupation/ffade2f4...,civil service administrative officer,2422,11-3011.00,administrative services managers,2,24,242,35.240566,33292.986333,0.001128,36261.0


left-join job zone data

In [130]:
df_merged_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2942 entries, 0 to 2941
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2942 non-null   int64  
 1   concept_uri       2942 non-null   object 
 2   preferred_label   2942 non-null   object 
 3   isco_level_4      2942 non-null   int64  
 4   onet_code         2921 non-null   object 
 5   onet_occupation   2921 non-null   object 
 6   isco_level_1      2942 non-null   int64  
 7   isco_level_2      2942 non-null   int64  
 8   isco_level_3      2942 non-null   int64  
 9   total_paid_hours  2926 non-null   float64
 10  annual_earnings   2926 non-null   float64
 11  employment_share  2857 non-null   float64
 12  employment_count  2857 non-null   float64
dtypes: float64(4), int64(5), object(4)
memory usage: 298.9+ KB


In [128]:
df_merged_4 = pd.merge(
    left=df_merged_3,
    right=job_zones[["isco_level_4", "job_zone", "education_level", "related_work_experience", "on_the_job_training"]],
    on="isco_level_4",
    how="left"
).drop_duplicates()

df_merged_4

,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_occupation,isco_level_1,isco_level_2,isco_level_3,total_paid_hours,annual_earnings,employment_share,employment_count,job_zone,education_level,related_work_experience,on_the_job_training
0,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0,4.0,6.2477,8.0309,4.3843
1,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0,4.0,5.2377,6.2857,2.9999
3,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0,4.0,4.9737,6.4292,2.4731
11,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0,4.0,5.3847,5.9240,3.0385
13,0,http://data.europa.eu/esco/occupation/00030d09...,technical director,2166,27-1011.00,art directors,2,21,216,36.670202,37555.308176,0.000294,9452.0,3.0,5.0687,6.7961,2.6492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52318,2940,http://data.europa.eu/esco/occupation/ffade2f4...,civil service administrative officer,2422,11-3011.00,administrative services managers,2,24,242,35.240566,33292.986333,0.001128,36261.0,3.0,3.2500,5.8000,4.2000
52322,2940,http://data.europa.eu/esco/occupation/ffade2f4...,civil service administrative officer,2422,11-3011.00,administrative services managers,2,24,242,35.240566,33292.986333,0.001128,36261.0,3.0,4.4890,6.6837,3.1881
52326,2940,http://data.europa.eu/esco/occupation/ffade2f4...,civil service administrative officer,2422,11-3011.00,administrative services managers,2,24,242,35.240566,33292.986333,0.001128,36261.0,3.0,3.9636,6.9711,3.4248
52327,2941,http://data.europa.eu/esco/occupation/ffc292f1...,motor vehicle assembler,8211,51-2031.00,engine and other machine assemblers,8,82,821,41.487179,29879.320665,0.000167,5356.0,2.0,3.0625,5.3635,4.0223


In [110]:
df_merged_3.to_csv(
    os.path.join(data_interim, "ESCO_ONET_xwalk_full_merged_metadata.csv")
)